In [1]:
import pandas as pd

# For a tab-separated file

# For a comma-separated file (often also labeled as `.txt`)
df = pd.read_csv("agency.txt", delimiter=",")
print(df.shape)
df.head()


(1, 6)


,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,STIB/MIVB,STIB,https://www.stib-mivb.be/index.htm?l=fr,Europe/Brussels,fr,070 23 2000


In [2]:
df = pd.read_csv("calendar_dates.txt", delimiter=",")
print(df.shape)
df.head()


(415, 3)


,service_id,date,exception_type
0,285170074,20241128,2
1,285170074,20241129,2
2,285170074,20241202,2
3,285170074,20241203,2
4,285160009,20241127,2


In [3]:
df = pd.read_csv("calendar.txt", delimiter=",")
print(df.shape)
df.head()


(303, 10)


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,281930506,0,0,0,0,0,1,0,20241130,20241221
1,281793005,1,1,1,1,1,0,0,20241125,20241220
2,281932610,0,0,0,0,0,0,1,20241201,20241222
3,277026001,1,1,1,1,1,0,0,20241125,20241220
4,276683501,0,0,0,0,0,1,0,20241130,20241221


In [4]:
df = pd.read_csv("routes.txt", delimiter=",")
print(df.shape)
df.head()


(91, 8)


,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,B5378C,FFFFFF
1,3,2,SIMONIS - ELISABETH,NaN,1,NaN,ED6C23,FFFFFF
2,5,4,GARE DU NORD - STALLE (P),NaN,0,NaN,EA4F80,000000
3,2,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,F6A90B,FFFFFF
4,4,6,ROI BAUDOUIN - ELISABETH,NaN,1,NaN,0066A3,FFFFFF


In [5]:
df = pd.read_csv("stop_times.txt", delimiter=",")
print(df.shape)
df.head()


(1272009, 7)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,123006309281795003,22:46:00,22:46:00,1780,1,0,0
1,123006309281795003,22:47:00,22:47:00,6433,2,0,0
2,123006309281795003,22:49:00,22:49:00,1131B,3,0,0
3,123006309281795003,22:53:00,22:53:00,6464,4,0,0
4,123006309281795003,22:57:00,22:57:00,1318,5,0,0


In [6]:
df = pd.read_csv("stops.txt", delimiter=",")
print(df.shape)
df.head()


(2803, 10)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0089,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,13.0
1,0471,NaN,SIMONIS,NaN,50.863732,4.329236,NaN,NaN,0,38.0
2,0472,NaN,SIMONIS,NaN,50.863543,4.329023,NaN,NaN,0,38.0
3,0501,NaN,DE BROUCKERE,NaN,50.851494,4.352876,NaN,NaN,0,6.0
4,0506,NaN,DE BROUCKERE,NaN,50.851746,4.353415,NaN,NaN,0,6.0


In [11]:
df = pd.read_csv("trips.txt", delimiter=",")
print(df.shape)
df.head()

(62221, 7)


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,16,281795003,123006309281795003,AIRPORT VIA MAEL,1,9682683,012b0010
1,16,281795003,123006310281795003,CITY VIA MAELB.,0,9682679,012b0011
2,16,281795003,123006311281795003,CITY VIA MAELB.,0,9682683,012b0011
3,16,281795003,123006312281795003,CITY VIA MAELB.,0,9682667,012b0011
4,16,281795003,123006313281795003,AIRPORT VIA MAEL,1,9682667,012b0010


In [12]:
import pandas as pd

# Load GTFS files
routes_df = pd.read_csv("routes.txt")
trips_df = pd.read_csv("trips.txt")
calendar_df = pd.read_csv("calendar.txt")

try:
    calendar_dates_df = pd.read_csv("calendar_dates.txt")
except FileNotFoundError:
    calendar_dates_df = None

# Step 1: Calculate weekly service days for each service_id
# Sum the number of active days (Mon-Sun) for each service_id in calendar.txt
calendar_df['weekly_active_days'] = calendar_df[['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']].sum(axis=1)

# Step 2: Merge trips with calendar to link trips to service days
trips_with_service = pd.merge(trips_df, calendar_df[['service_id', 'weekly_active_days']], on='service_id', how='left')

# Step 3: Compute weekly trips per route
# Multiply the number of weekly active days by the number of trips for each route_id
trips_with_service['weekly_trips'] = trips_with_service['weekly_active_days']

# Group by route_id and sum up weekly trips
weekly_trips_per_route = trips_with_service.groupby('route_id')['weekly_trips'].sum().reset_index()

# Step 4: Merge with route details for human-readable output
weekly_trips_summary = pd.merge(weekly_trips_per_route, routes_df, on='route_id', how='left')

# Display result
print(weekly_trips_summary[['route_id', 'route_short_name', 'route_long_name', 'weekly_trips']])

# Save to CSV (optional)
weekly_trips_summary.to_csv("weekly_trips_per_route.csv", index=False)


    route_id route_short_name                  route_long_name  weekly_trips
0          1                1        GARE DE L'OUEST - STOCKEL          2261
1          2                5        ERASME - HERRMANN-DEBROUX          2332
2          3                2              SIMONIS - ELISABETH          2123
3          4                6         ROI BAUDOUIN - ELISABETH          2181
4          5                4        GARE DU NORD - STALLE (P)          2267
..       ...              ...                              ...           ...
86        87              240          CERIA - FOREST NATIONAL            68
87        88               T8               LOUISE - ROODEBEEK          1704
88        89              T44    MONTGOMERY - TERVUREN STATION          1176
89        90              T92      SCHAERBEEK GARE - FORT-JACO          1757
90        91               T9  GROOT-BIJGAARDEN - ROI BAUDOUIN          2694

[91 rows x 4 columns]


In [17]:
import pandas as pd
import numpy as np

# Load required GTFS files
stops_df = pd.read_csv("stops.txt")
stop_times_df = pd.read_csv("stop_times.txt")

# Step 1: Merge stop_times with trips to associate trips with service days
stop_times_with_trips = pd.merge(stop_times_df, trips_with_service[['trip_id', 'weekly_trips']], on='trip_id', how='left')

# Step 2: Create edges between consecutive stops in stop_times
stop_times_with_trips.sort_values(by=['trip_id', 'stop_sequence'], inplace=True)
stop_times_with_trips['next_stop_id'] = stop_times_with_trips.groupby('trip_id')['stop_id'].shift(-1)

# Drop rows where next_stop_id is NaN (last stop in each trip)
stop_times_with_trips = stop_times_with_trips.dropna(subset=['next_stop_id'])

# Step 3: Sum weights for edges
# Each edge (stop_id -> next_stop_id) will have a weight equal to the sum of weekly trips
edges = stop_times_with_trips.groupby(['stop_id', 'next_stop_id'])['weekly_trips'].sum().reset_index()

# Step 4: Create adjacency matrix
# Create a mapping of stop_id to matrix index
stop_ids = pd.concat([edges['stop_id'], edges['next_stop_id']]).unique()
stop_id_to_index = {stop_id: i for i, stop_id in enumerate(stop_ids)}

# Initialize adjacency matrix
adj_matrix = np.zeros((len(stop_ids), len(stop_ids)))

# Populate adjacency matrix with weights
for _, row in edges.iterrows():
    i = stop_id_to_index[row['stop_id']]
    j = stop_id_to_index[row['next_stop_id']]
    adj_matrix[i, j] = row['weekly_trips']

# Convert to DataFrame for easier interpretation (optional)
adj_matrix_df = pd.DataFrame(adj_matrix, index=stop_ids, columns=stop_ids)

# Save adjacency matrix to a file (optional)
adj_matrix_df.to_csv("adjacency_matrix.csv")

# Print adjacency matrix
print(adj_matrix_df)


      0089  0472    0501  0506  0511    0516  0521  0526  0529  0531  ...  \
0089   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
0472   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
0501   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
0506   0.0   0.0     0.0   0.0   0.0  2016.0   0.0   0.0   0.0   0.0  ...   
0511   0.0   0.0  2029.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
...    ...   ...     ...   ...   ...     ...   ...   ...   ...   ...  ...   
5320   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
4265   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
1390   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
7305   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   
8472   0.0   0.0     0.0   0.0   0.0     0.0   0.0   0.0   0.0   0.0  ...   

      5776  6486G  0712F  4502F  9952G  5320  4265  1390  7305  8472  
0089

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

# Load adjacency matrix
adj_matrix_df = pd.read_csv("adjacency_matrix.csv", index_col=0)

# Convert adjacency matrix to a numpy array
adj_matrix = adj_matrix_df.values

# Create a directed graph from the adjacency matrix
G = nx.from_numpy_matrix(adj_matrix, create_using=nx.DiGraph)

# Relabel nodes with stop IDs for clarity (optional)
mapping = {i: stop_id for i, stop_id in enumerate(adj_matrix_df.index)}
G = nx.relabel_nodes(G, mapping)

# Calculate eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality_numpy(G, weight='weight')

# Convert results to a DataFrame for easier interpretation
eigenvector_centrality_df = pd.DataFrame(
    list(eigenvector_centrality.items()), 
    columns=['stop_id', 'eigenvector_centrality']
)

# Sort by centrality (descending order)
eigenvector_centrality_df = eigenvector_centrality_df.sort_values(by='eigenvector_centrality', ascending=False)

# Save to CSV (optional)
eigenvector_centrality_df.to_csv("eigenvector_centrality.csv", index=False)

# Print the top stops by eigenvector centrality
print(eigenvector_centrality_df.head())


In [21]:
import pandas as pd
import numpy as np
import networkx as nx

# Load adjacency matrix
adj_matrix_df = pd.read_csv("adjacency_matrix.csv", index_col=0)

# Convert adjacency matrix to a numpy array
adj_matrix = adj_matrix_df.values

# Create a directed graph from the adjacency matrix
G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)

# Relabel nodes with stop IDs for clarity (optional)
mapping = {i: stop_id for i, stop_id in enumerate(adj_matrix_df.index)}
G = nx.relabel_nodes(G, mapping)

# Calculate eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality_numpy(G, weight='weight')

# Convert results to a DataFrame for easier interpretation
eigenvector_centrality_df = pd.DataFrame(
    list(eigenvector_centrality.items()), 
    columns=['stop_id', 'eigenvector_centrality']
)

# Sort by centrality (descending order)
eigenvector_centrality_df = eigenvector_centrality_df.sort_values(by='eigenvector_centrality', ascending=False)

# Save to CSV (optional)
eigenvector_centrality_df.to_csv("eigenvector_centrality.csv", index=False)

# Print the top stops by eigenvector centrality
print(eigenvector_centrality_df.head())


     stop_id  eigenvector_centrality
1103    2928                0.250939
1901    5611                0.239206
1315    3372                0.226240
1374    3517                0.213970
1370   3510B                0.202366


In [25]:
!pip install python-louvain

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9402 sha256=b1886ba74c37d66cfcc6210bceae40df5dc5038c9273a9fdbb21caaf0b3d06a0
  Stored in directory: c:\users\padidar\appdata\local\pip\cache\wheels\40\f1\e3\485b698c520fa0baee1d07897abc7b8d6479b7d199ce96f4af
Successfully built python-louvain


In [27]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import community

# Load adjacency matrix
adj_matrix_df = pd.read_csv("adjacency_matrix.csv", index_col=0)

# Convert adjacency matrix to a numpy array
adj_matrix = adj_matrix_df.values

# Create a directed graph from the adjacency matrix
G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)

# Relabel nodes with stop IDs for clarity (optional)
mapping = {i: stop_id for i, stop_id in enumerate(adj_matrix_df.index)}
G = nx.relabel_nodes(G, mapping)

# Detect communities using the Louvain method
from community import community_louvain  # Requires `python-louvain` package
partition = community_louvain.best_partition(G.to_undirected(), weight='weight')

# Calculate modularity
modularity = community_louvain.modularity(partition, G.to_undirected(), weight='weight')

# Print modularity value
print(f"Modularity: {modularity}")

# Save community assignments to a file (optional)
partition_df = pd.DataFrame(list(partition.items()), columns=['stop_id', 'community'])
partition_df.to_csv("communities.csv", index=False)

# Print the detected communities
print(partition_df.head())


Modularity: 0.9222544413409264
  stop_id  community
0    0089          0
1    0472          1
2    0501          2
3    0506          2
4    0511          2


In [29]:
import pandas as pd
import numpy as np
import networkx as nx

# Load adjacency matrix
adj_matrix_df = pd.read_csv("adjacency_matrix.csv", index_col=0)

# Convert adjacency matrix to a numpy array
adj_matrix = adj_matrix_df.values

# Create a graph (undirected for cliques)
G = nx.from_numpy_array(adj_matrix)

# Relabel nodes with stop IDs for clarity (optional)
mapping = {i: stop_id for i, stop_id in enumerate(adj_matrix_df.index)}
G = nx.relabel_nodes(G, mapping)

# Find all cliques
all_cliques = list(nx.find_cliques(G))

# Find the maximum clique
max_clique = max(all_cliques, key=len)

# Print results
print(f"All Cliques: {all_cliques}")
print(f"Maximum Clique: {max_clique}")
print(f"Size of Maximum Clique: {len(max_clique)}")

# Save all cliques to a file (optional)
cliques_df = pd.DataFrame({'clique': all_cliques})
cliques_df.to_csv("cliques.csv", index=False)


All Cliques: [['4014', '3231'], ['4014', '3282'], ['1064', '1111'], ['1064', '1094F'], ['3910', '1557'], ['3910', '3909'], ['3910', '3919'], ['3910', '1578'], ['2334B', '2404'], ['2334B', '2402'], ['4124', '4131B'], ['4124', '4125'], ['5838F', '5836F'], ['5838F', '0712F'], ['8733', '8742'], ['3257F', '3263F'], ['3257F', '3253F'], ['5287', '1816'], ['5287', '1818'], ['8824', '8834'], ['8824', '8833'], ['8824', '8814'], ['9660', '9654'], ['9660', '9652'], ['7474', '7472'], ['7474', '3405'], ['1490', '2375'], ['1490', '5296B'], ['1178', '4595'], ['1178', '2782'], ['2788', '2739'], ['2788', '2221'], ['2788', '2296'], ['2788', '1000', '1001'], ['2855', '2856B'], ['2855', '2854'], ['1829', '1830'], ['1829', '1828'], ['3858', '1665'], ['3858', '3852'], ['1233', '1706'], ['1233', '1729'], ['1233', '1906'], ['6359F', '6434F'], ['6359F', '6413F'], ['6359F', '6409F'], ['3360F', '1698F'], ['3360F', '6204F'], ['1651', '1943'], ['1651', '1650'], ['1863', '1864'], ['1863', '1862'], ['1871B', '1872B']